1. Titre du Projet
Harmonisation Automatique des Noms de Clients avec le Machine Learning

2. Méthodologie Détaillée

Étape 1 : Compréhension et Préparation des Données

Collecte des Données : Importer le fichier contenant les données des clients, s'assurer que la colonne des noms de clients est bien identifiée.
Exploration des Données : Analyser les différentes variantes des noms de clients pour comprendre la nature et l'étendue du problème. Identifier les patterns communs et les anomalies.
Prétraitement des Données : Nettoyer les données pour éliminer les doublons, les valeurs manquantes, et toute autre anomalie apparente.

Étape 2 : Extraction des Caractéristiques

Normalisation : Appliquer des techniques de normalisation sur les noms pour les uniformiser. Cela inclut :
Conversion en minuscules
Suppression des accents et des caractères spéciaux
Normalisation des espaces
Tri des mots dans les noms pour gérer les permutations
Tokenization : Diviser chaque nom en tokens (prénoms, noms de famille, initiales, etc.).
Encodage des Caractéristiques : Transformer les tokens en vecteurs numériques à l'aide de techniques comme TF-IDF ou des embeddings de mots (Word2Vec, GloVe).

Étape 3 : Modélisation

Clustering : Utiliser des algorithmes de clustering comme DBSCAN pour regrouper les noms similaires. DBSCAN est choisi pour sa capacité à identifier des clusters de formes variées et à gérer le bruit.
Modèle de Correspondance : Entraîner un modèle de correspondance de noms, tel qu'un réseau de neurones ou un modèle basé sur des distances (cosine similarity, Jaccard index, etc.), pour identifier les noms similaires.
Supervised Learning : Si des étiquettes de noms harmonisés sont disponibles, utiliser des algorithmes supervisés comme les forêts aléatoires, les réseaux de neurones ou les SVMs pour apprendre les patterns de correspondance.

Étape 4 : Post-Traitement et Validation

Regroupement et Validation : Regrouper les variantes identifiées par le modèle. Vérifier manuellement un échantillon des résultats pour s'assurer de la précision du modèle.
Évaluation du Modèle : Utiliser des métriques d'évaluation comme la précision, le rappel, le F1-score pour mesurer la performance du modèle. Ajuster les paramètres du modèle si nécessaire pour améliorer les résultats.
Automatisation du Processus : Une fois le modèle validé, automatiser le processus d'harmonisation des noms pour qu'il puisse être appliqué à de nouveaux ensembles de données sans intervention manuelle.

Étape 5 : Implémentation et Suivi

Déploiement : Déployer le modèle dans un environnement de production, où il peut traiter les nouvelles entrées de données en temps réel ou par lots.
Monitoring : Mettre en place un système de suivi pour surveiller la performance du modèle au fil du temps et détecter toute dégradation de performance.
Mise à Jour : Prévoir des mises à jour régulières du modèle pour inclure de nouvelles variantes de noms ou pour ajuster aux évolutions des données.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import DBSCAN
from sklearn.metrics.pairwise import cosine_similarity

# Exemple de données avec plus de variations
data = {
    'client_names': [
        'KOUASSI Franck Ismael', 'KOUASSI Ismael Franck', 'Ismael Franck KOUASSI',
        'Franck Ismael KOUASSI', 'KOUASI Franck Ismael', 'KOUSSI Franck Ismael',
        'KOUASSI Frack Ismael', 'KOUASSI Franck Ismal', 'Franck KOUASSI',
        'Ismael KOUASSI', 'KOUASSI I. Franck'
    ]
}

# Chargement des données dans un DataFrame
df = pd.DataFrame(data)

# Prétraitement : normalisation des noms
def normalize_name(name):
    name = name.lower()  # Mettre en minuscules
    name = name.replace('.', '')  # Supprimer les points
    name = name.replace(',', '')  # Supprimer les virgules
    name = name.replace('é', 'e')  # Normaliser les accents
    name = name.replace('è', 'e')  # Normaliser les accents
    name = ' '.join(sorted(name.split()))  # Trier les mots dans l'ordre alphabétique
    return name

df['normalized_names'] = df['client_names'].apply(normalize_name)

# Vectorisation TF-IDF des noms normalisés
vectorizer = TfidfVectorizer().fit_transform(df['normalized_names'])
vectors = vectorizer.toarray()

# Clustering DBSCAN basé sur la similarité cosinus
dbscan = DBSCAN(metric='cosine', eps=0.1, min_samples=1)
df['cluster'] = dbscan.fit_predict(vectors)

# Fonction pour obtenir le nom le plus fréquent dans chaque cluster
def get_representative_name(cluster_df):
    return cluster_df['client_names'].mode()[0]

# Harmonisation des noms par cluster
harmonized_names = df.groupby('cluster').apply(get_representative_name)
harmonized_names = harmonized_names.to_dict()

# Appliquer les noms harmonisés au DataFrame original
df['harmonized_name'] = df['cluster'].map(harmonized_names)

# Affichage des résultats
print(df[['client_names', 'harmonized_name']])

In [ ]:
pip install textdistance pandas scikit-learn

In [ ]:
import pandas as pd
import textdistance
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import DBSCAN

# Exemple de données avec plus de variations
data = {
    'client_names': [
        'KOUASSI Franck Ismael', 'KOUASSI Ismael Franck', 'Ismael Franck KOUASSI',
        'Franck Ismael KOUASSI', 'KOUASI Franck Ismael', 'KOUSSI Franck Ismael',
        'KOUASSI Frack Ismael', 'KOUASSI Franck Ismal', 'Franck KOUASSI',
        'Ismael KOUASSI', 'KOUASSI I. Franck'
    ]
}

# Chargement des données dans un DataFrame
df = pd.DataFrame(data)

# Dictionnaire de correction manuelle des erreurs
correction_dict = {
    'frack': 'franck',
    'ismal': 'ismael'
}

# Fonction de correction des erreurs de frappe
def correct_spelling(name, correction_dict):
    words = name.split()
    corrected_words = []
    for word in words:
        # Trouver le mot le plus similaire dans le dictionnaire de correction
        corrected_word = min(correction_dict.keys(), key=lambda x: textdistance.levenshtein(word.lower(), x))
        # Si la distance de Levenshtein est inférieure à un seuil, corriger le mot
        if textdistance.levenshtein(word.lower(), corrected_word) <= 2:
            corrected_words.append(correction_dict[corrected_word])
        else:
            corrected_words.append(word)
    return ' '.join(corrected_words)

# Correction des erreurs dans les noms
df['corrected_names'] = df['client_names'].apply(lambda x: correct_spelling(x, correction_dict))

# Prétraitement : normalisation des noms
def normalize_name(name):
    name = name.lower()  # Mettre en minuscules
    name = name.replace('.', '')  # Supprimer les points
    name = name.replace(',', '')  # Supprimer les virgules
    name = name.replace('é', 'e')  # Normaliser les accents
    name = name.replace('è', 'e')  # Normaliser les accents
    name = ' '.join(sorted(name.split()))  # Trier les mots dans l'ordre alphabétique
    return name

df['normalized_names'] = df['corrected_names'].apply(normalize_name)

# Vectorisation TF-IDF des noms normalisés
vectorizer = TfidfVectorizer().fit_transform(df['normalized_names'])
vectors = vectorizer.toarray()

# Clustering DBSCAN basé sur la similarité cosinus
dbscan = DBSCAN(metric='cosine', eps=0.1, min_samples=1)
df['cluster'] = dbscan.fit_predict(vectors)

# Fonction pour obtenir le nom le plus fréquent dans chaque cluster
def get_representative_name(cluster_df):
    return cluster_df['client_names'].mode()[0]

# Harmonisation des noms par cluster
harmonized_names = df.groupby('cluster').apply(get_representative_name)
harmonized_names = harmonized_names.to_dict()

# Appliquer les noms harmonisés au DataFrame original
df['harmonized_name'] = df['cluster'].map(harmonized_names)

# Affichage des résultats
print(df[['client_names', 'corrected_names', 'harmonized_name']])

In [1]:
import pandas as pd
import textdistance
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import DBSCAN

# Exemple de données avec plus de variations
data = {
    'client_names': [
        'KOUASSI Franck Ismael', 'KOUASSI Ismael Franck', 'Ismael Franck KOUASSI',
        'Franck Ismael KOUASSI', 'KOUASI Franck Ismael', 'KOUSSI Franck Ismael',
        'KOUASSI Frack Ismael', 'KOUASSI Franck Ismal', 'Franck KOUASSI',
        'Ismael KOUASSI', 'KOUASSI I. Franck'
    ]
}

# Chargement des données dans un DataFrame
df = pd.DataFrame(data)

# Dictionnaire de correction manuelle des erreurs
correction_dict = {
    'frack': 'franck',
    'ismal': 'ismael'
}

# Fonction de correction des erreurs de frappe
def correct_spelling(name, correction_dict):
    words = name.split()
    corrected_words = []
    for word in words:
        # Trouver le mot le plus similaire dans le dictionnaire de correction
        corrected_word = min(correction_dict.keys(), key=lambda x: textdistance.levenshtein(word.lower(), x))
        # Si la distance de Levenshtein est inférieure à un seuil, corriger le mot
        if textdistance.levenshtein(word.lower(), corrected_word) <= 2:
            corrected_words.append(correction_dict[corrected_word])
        else:
            corrected_words.append(word)
    return ' '.join(corrected_words)

# Correction des erreurs dans les noms
df['corrected_names'] = df['client_names'].apply(lambda x: correct_spelling(x, correction_dict))

# Prétraitement : normalisation des noms
def normalize_name(name):
    name = name.lower()  # Mettre en minuscules
    name = name.replace('.', '')  # Supprimer les points
    name = name.replace(',', '')  # Supprimer les virgules
    name = name.replace('é', 'e')  # Normaliser les accents
    name = name.replace('è', 'e')  # Normaliser les accents
    name = ' '.join(sorted(name.split()))  # Trier les mots dans l'ordre alphabétique
    return name

df['normalized_names'] = df['corrected_names'].apply(normalize_name)

# Vectorisation TF-IDF des noms normalisés
vectorizer = TfidfVectorizer().fit_transform(df['normalized_names'])
vectors = vectorizer.toarray()

# Clustering DBSCAN basé sur la similarité cosinus
dbscan = DBSCAN(metric='cosine', eps=0.1, min_samples=1)
df['cluster'] = dbscan.fit_predict(vectors)

# Fonction pour obtenir le nom le plus fréquent dans chaque cluster
def get_representative_name(cluster_df):
    return cluster_df['client_names'].mode()[0]

# Harmonisation des noms par cluster
harmonized_names = df.groupby('cluster').apply(get_representative_name)
harmonized_names = harmonized_names.to_dict()

# Appliquer les noms harmonisés au DataFrame original
df['harmonized_name'] = df['cluster'].map(harmonized_names)

# Affichage des résultats
print(df[['client_names', 'corrected_names', 'harmonized_name']])

             client_names        corrected_names        harmonized_name
0   KOUASSI Franck Ismael  KOUASSI franck ismael  Franck Ismael KOUASSI
1   KOUASSI Ismael Franck  KOUASSI ismael franck  Franck Ismael KOUASSI
2   Ismael Franck KOUASSI  ismael franck KOUASSI  Franck Ismael KOUASSI
3   Franck Ismael KOUASSI  franck ismael KOUASSI  Franck Ismael KOUASSI
4    KOUASI Franck Ismael   KOUASI franck ismael   KOUASI Franck Ismael
5    KOUSSI Franck Ismael   KOUSSI franck ismael   KOUSSI Franck Ismael
6    KOUASSI Frack Ismael  KOUASSI franck ismael  Franck Ismael KOUASSI
7    KOUASSI Franck Ismal  KOUASSI franck ismael  Franck Ismael KOUASSI
8          Franck KOUASSI         franck KOUASSI         Franck KOUASSI
9          Ismael KOUASSI         ismael KOUASSI         Ismael KOUASSI
10      KOUASSI I. Franck      KOUASSI I. franck         Franck KOUASSI


In [3]:
pip install fuzzywuzzy

  Obtaining dependency information for fuzzywuzzy from https://files.pythonhosted.org/packages/43/ff/74f23998ad2f93b945c0309f825be92e04e0348e062026998b5eefef4c33/fuzzywuzzy-0.18.0-py2.py3-none-any.whl.metadata
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import DBSCAN
from fuzzywuzzy import fuzz
import nltk

# Prétraitement
def preprocess_name(name):
    name = name.lower()
    name = nltk.word_tokenize(name)
    name = sorted(name)
    return " ".join(name)

# Similarité basée sur FuzzyWuzzy
def calculate_similarity(name1, name2):
    return fuzz.token_sort_ratio(name1, name2)

# Charger les noms
names = [
    "KOUASSI Franck Ismael", "KOUASSI Ismael Franck", "Ismael Franck KOUASSI",
    "Franck Ismael KOUASSI", "KOUASI Franck Ismael", "KOUSSI Franck Ismael",
    "KOUASSI Frack Ismael", "KOUASSI Franck Ismal", "Franck KOUASSI", "Ismael KOUASSI", "KOUASSI I. Franck"
]

# Prétraiter les noms
preprocessed_names = [preprocess_name(name) for name in names]

# Vectorisation TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(preprocessed_names)

# Clustering avec DBSCAN
dbscan = DBSCAN(eps=0.5, min_samples=2, metric='cosine')
clusters = dbscan.fit_predict(X)

# Résultats
name_clusters = pd.DataFrame({'name': names, 'cluster': clusters})
print(name_clusters)

/Users/macbookpro/anaconda3/lib/python3.11/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/PY3/english.pickle[0m

  Searched in:
    - '/Users/macbookpro/nltk_data'
    - '/Users/macbookpro/anaconda3/nltk_data'
    - '/Users/macbookpro/anaconda3/share/nltk_data'
    - '/Users/macbookpro/anaconda3/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************
